# Importing Library and Loading Dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

In [4]:
df=pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

x_train.shape

(80, 5)

In [9]:
x_train.head()

,age,gender,fever,cough,city
22,71,Female,98.0,Strong,Kolkata
88,5,Female,100.0,Mild,Kolkata
61,81,Female,98.0,Strong,Mumbai
8,19,Female,100.0,Strong,Bangalore
35,82,Female,102.0,Strong,Bangalore


# Method 1: Without Column Trasformer

## 1. Ordinal Encoding

In [14]:
oe=OrdinalEncoder(categories=[['Mild','Strong']],dtype=np.int32)

x_train_cough=oe.fit_transform(x_train[['cough']])
x_test_cough=oe.transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

## 2. One Hot Encoding

In [32]:
ohe=OneHotEncoder(drop='first',sparse_output=False)

x_train_gender_city=ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city=ohe.transform(x_test[['gender','city']])
    
x_train_gender_city.shape


(80, 4)

## 3. Simple Imputer 

In [34]:
si=SimpleImputer()

x_train_fever=si.fit_transform(x_train[['fever']])
x_test_fever=si.transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

## 4. Label Encoding 

In [43]:
le=LabelEncoder()

y_train_trans=le.fit_transform(y_train)
y_test_trans=le.transform(y_test)

y_train_trans.shape

(80,)

## 5. Extracting age

In [55]:
x_train_age = x_train[['age']].values
x_test_age = x_test[['age']].values

x_train_age.shape

(80, 1)

## 6. Concatinating all the data 

In [58]:
x_train_trans=np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)

x_test_trans=np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_trans.shape

(80, 7)

In [59]:

x_train_trans_df = pd.DataFrame(x_train_trans)
x_test_trans_df = pd.DataFrame(x_test_trans)


In [60]:
x_train_trans_df.head()

,0,1,2,3,4,5,6
0,71.0,98.0,0.0,0.0,1.0,0.0,1.0
1,5.0,100.0,0.0,0.0,1.0,0.0,0.0
2,81.0,98.0,0.0,0.0,0.0,1.0,1.0
3,19.0,100.0,0.0,0.0,0.0,0.0,1.0
4,82.0,102.0,0.0,0.0,0.0,0.0,1.0


In [63]:
column_names = ['Age', 'Fever', 'Gender_Male', 'City_Delhi', 'City_Kolkata', 'City_Mumbai', 'Cough']

x_train_trans_df = pd.DataFrame(x_train_trans, columns=column_names)
x_test_trans_df = pd.DataFrame(x_test_trans, columns=column_names)

In [66]:
x_train_trans_df.sample(5)

,Age,Fever,Gender_Male,City_Delhi,City_Kolkata,City_Mumbai,Cough
45,82.0,100.859155,1.0,0.0,1.0,0.0,0.0
65,38.0,100.859155,1.0,1.0,0.0,0.0,0.0
16,16.0,104.000000,1.0,0.0,1.0,0.0,0.0
77,10.0,100.000000,1.0,0.0,0.0,0.0,0.0
41,60.0,103.000000,1.0,0.0,1.0,0.0,0.0


In [65]:
x_train

,age,gender,fever,cough,city
22,71,Female,98.0,Strong,Kolkata
88,5,Female,100.0,Mild,Kolkata
61,81,Female,98.0,Strong,Mumbai
8,19,Female,100.0,Strong,Bangalore
35,82,Female,102.0,Strong,Bangalore
...,...,...,...,...,...
16,69,Female,103.0,Mild,Kolkata
33,26,Female,98.0,Mild,Kolkata
63,10,Male,100.0,Mild,Bangalore
96,51,Female,101.0,Strong,Kolkata


# Method 2: using Column Transformer

In [68]:
from sklearn.compose import ColumnTransformer

In [69]:
transformer=ColumnTransformer(
    transformers=[
        ('tnf1',SimpleImputer(),['fever']),
        ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
        ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
    ],remainder='passthrough'
)

In [70]:
transformer.fit(x_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [73]:
x_train_trans=transformer.transform(x_train)
x_test_trans=transformer.transform(x_test)


In [74]:
column_names = ['fever', 'cough', 'Gender_Male', 'City_Delhi', 'City_Kolkata', 'City_Mumbai', 'age']

x_train_trans_df1 = pd.DataFrame(x_train_trans, columns=column_names)
x_test_trans_df1 = pd.DataFrame(x_test_trans, columns=column_names)

In [75]:
x_train_trans_df1.head()

,fever,cough,Gender_Male,City_Delhi,City_Kolkata,City_Mumbai,age
0,98.0,1.0,0.0,0.0,1.0,0.0,71.0
1,100.0,0.0,0.0,0.0,1.0,0.0,5.0
2,98.0,1.0,0.0,0.0,0.0,1.0,81.0
3,100.0,1.0,0.0,0.0,0.0,0.0,19.0
4,102.0,1.0,0.0,0.0,0.0,0.0,82.0


In [76]:
x_train_trans_df.head()

,Age,Fever,Gender_Male,City_Delhi,City_Kolkata,City_Mumbai,Cough
0,71.0,98.0,0.0,0.0,1.0,0.0,1.0
1,5.0,100.0,0.0,0.0,1.0,0.0,0.0
2,81.0,98.0,0.0,0.0,0.0,1.0,1.0
3,19.0,100.0,0.0,0.0,0.0,0.0,1.0
4,82.0,102.0,0.0,0.0,0.0,0.0,1.0
